# Big Mart Sales Prediction

This problem was discovered on the Interet and the dataset is available in the datasets folder on my page. I decided to create sales volume buckets for this problem instead of the typical regression approach. These buckets where then converted into numerical values for prediction purposes. 

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn import metrics
from sklearn import model_selection

Here is where the dataset is identified and loaded into a dataframe.

In [30]:
dataset=r'C:\Users\claudius.ibine\Desktop\big-mart-dataset.csv'
dataset=pd.read_csv(dataset,index_col=None)

Here is where I summarized the dataset.

In [31]:
def aggregrate():
    print('The number of rows by columns:  {0}'.format(dataset.shape))
    print("\n"+'The first 10 records are:')
    print(dataset.head(5))
    print("\n"+'The dataset datatypes are:')
    print(dataset.dtypes)

In [32]:
aggregrate()

The number of rows by columns:  (8523, 12)

The first 10 records are:
  Item_Identifier  Item_Weight Item_Fat_Content  Item_Visibility  \
0           FDA15         9.30          Low Fat         0.016047   
1           DRC01         5.92          Regular         0.019278   
2           FDN15        17.50          Low Fat         0.016760   
3           FDX07        19.20          Regular         0.000000   
4           NCD19         8.93          Low Fat         0.000000   

               Item_Type  Item_MRP Outlet_Identifier  \
0                  Dairy  249.8092            OUT049   
1            Soft Drinks   48.2692            OUT018   
2                   Meat  141.6180            OUT049   
3  Fruits and Vegetables  182.0950            OUT010   
4              Household   53.8614            OUT013   

   Outlet_Establishment_Year Outlet_Size Outlet_Location_Type  \
0                       1999      Medium               Tier 1   
1                       2009      Medium              

Here is where I cleaned the dataset.

In [33]:
def clean():
    print(dataset.isnull().sum()) #check nulll values
    print(dataset.describe()) #check for low values in min
    print(dataset.head(10))

In [24]:
clean()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64
       Item_Weight  Item_Visibility     Item_MRP  Outlet_Establishment_Year  \
count  7060.000000      8523.000000  8523.000000                8523.000000   
mean     12.857645         0.066132   140.992782                1997.831867   
std       4.643456         0.051598    62.275067                   8.371760   
min       4.555000         0.000000    31.290000                1985.000000   
25%       8.773750         0.026989    93.826500                1987.000000   
50%      12.600000         0.053931   143.012800                1999.000000   
75%      16.850000        

In [34]:
dataset=dataset[['Item_Weight','Item_Fat_Content','Item_Type','Outlet_Identifier','Outlet_Establishment_Year','Outlet_Size','Outlet_Location_Type','Outlet_Type','Item_Outlet_Sales']]
dataset['Item_Weight'].fillna((dataset['Item_Weight'].mean()), inplace=True)
dataset['Outlet_Size'].fillna(dataset['Outlet_Size'].mode()[0], inplace=True)
dataset['weight_bin']=pd.cut(dataset['Item_Weight'],[0, 4, 8, 12,16, 20,40,60,80,100], labels=['0-4', '4-8', '8-12','12-16','16-20','20-40','40-60','60-80','80-100'])
dataset['sales_bin']=pd.cut(dataset['Item_Outlet_Sales'],[0, 3000, 6000, 9000,12000, 15000,20000], labels=['0-3000', '3000-6000', '6000-9000','9000-12000','12000-15000','15000-20000'])
dataset['Item_Fat_Content']=np.where(dataset['Item_Fat_Content']=='LF', 'Low Fat', dataset['Item_Fat_Content'])
dataset['Item_Fat_Content']=np.where(dataset['Item_Fat_Content']=='reg', 'Regular', dataset['Item_Fat_Content'])
dataset['Item_Fat_Content']=np.where(dataset['Item_Fat_Content']=='low fat', 'Low Fat', dataset['Item_Fat_Content'])
dataset['Outlet_Type']=np.where(dataset['Outlet_Type']=='Supermarket Type1', 'Supermarket', dataset['Outlet_Type'])
dataset['Outlet_Type']=np.where(dataset['Outlet_Type']=='Supermarket Type2', 'Supermarket', dataset['Outlet_Type'])
dataset['Outlet_Type']=np.where(dataset['Outlet_Type']=='Supermarket Type3', 'Supermarket', dataset['Outlet_Type'])
dataset['Item_Type']=np.where(dataset['Item_Type']=='Fruits and Vegetables', 'Food & Drinks', dataset['Item_Type'])
dataset['Item_Type']=np.where(dataset['Item_Type']=='Snack Foods', 'Food & Drinks', dataset['Item_Type'])
dataset['Item_Type']=np.where(dataset['Item_Type']=='Frozen Foods', 'Food & Drinks', dataset['Item_Type'])
dataset['Item_Type']=np.where(dataset['Item_Type']=='Dairy', 'Food & Drinks', dataset['Item_Type'])
dataset['Item_Type']=np.where(dataset['Item_Type']=='Canned', 'Food & Drinks', dataset['Item_Type'])
dataset['Item_Type']=np.where(dataset['Item_Type']=='Baking Goods', 'Food & Drinks', dataset['Item_Type'])
dataset['Item_Type']=np.where(dataset['Item_Type']=='Meat', 'Food & Drinks', dataset['Item_Type'])
dataset['Item_Type']=np.where(dataset['Item_Type']=='Hard Drinks', 'Food & Drinks', dataset['Item_Type'])
dataset['Item_Type']=np.where(dataset['Item_Type']=='Starchy Foods', 'Food & Drinks', dataset['Item_Type'])
dataset['Item_Type']=np.where(dataset['Item_Type']=='Breakfast', 'Food & Drinks', dataset['Item_Type'])
dataset['Item_Type']=np.where(dataset['Item_Type']=='Seafood', 'Food & Drinks', dataset['Item_Type'])
dataset['Item_Type']=np.where(dataset['Item_Type']=='Soft Drinks', 'Food & Drinks', dataset['Item_Type'])
dataset['Item_Type']=np.where(dataset['Item_Type']=='Breads', 'Food & Drinks', dataset['Item_Type'])
dataset['Item_Type']=np.where(dataset['Item_Type']=='Household', 'Non Consumables', dataset['Item_Type'])
dataset['Item_Type']=np.where(dataset['Item_Type']=='Health and Hygiene', 'Non Consumables', dataset['Item_Type'])
dataset=dataset[['Item_Weight','Item_Fat_Content','Item_Type','Outlet_Identifier','Outlet_Establishment_Year','Outlet_Size','Outlet_Location_Type','Outlet_Type','sales_bin','weight_bin']]

In [35]:
print(dataset)

      Item_Weight Item_Fat_Content        Item_Type Outlet_Identifier  \
0        9.300000          Low Fat    Food & Drinks            OUT049   
1        5.920000          Regular    Food & Drinks            OUT018   
2       17.500000          Low Fat    Food & Drinks            OUT049   
3       19.200000          Regular    Food & Drinks            OUT010   
4        8.930000          Low Fat  Non Consumables            OUT013   
5       10.395000          Regular    Food & Drinks            OUT018   
6       13.650000          Regular    Food & Drinks            OUT013   
7       12.857645          Low Fat    Food & Drinks            OUT027   
8       16.200000          Regular    Food & Drinks            OUT045   
9       19.200000          Regular    Food & Drinks            OUT017   
10      11.800000          Low Fat    Food & Drinks            OUT049   
11      18.500000          Regular    Food & Drinks            OUT046   
12      15.100000          Regular    Food & Drinks

Here I will convert the string values to numericals for the machine learning model.

In [36]:
def preprocess_dataset(df):
    processed_df=df.copy()
    le=preprocessing.LabelEncoder()
    processed_df.Item_Fat_Content = le.fit_transform(processed_df.Item_Fat_Content)
    processed_df.Item_Type = le.fit_transform(processed_df.Item_Type)
    processed_df.Outlet_Identifier = le.fit_transform(processed_df.Outlet_Identifier)
    processed_df.Outlet_Size = le.fit_transform(processed_df.Outlet_Size)
    processed_df.Outlet_Location_Type = le.fit_transform(processed_df.Outlet_Location_Type)
    processed_df.Outlet_Type = le.fit_transform(processed_df.Outlet_Type)
    processed_df.sales_bin = le.fit_transform(processed_df.sales_bin)
    processed_df.weight_bin = le.fit_transform(processed_df.weight_bin)
    return processed_df
dataset=preprocess_dataset(dataset)

In [37]:
print(dataset)

      Item_Weight  Item_Fat_Content  Item_Type  Outlet_Identifier  \
0        9.300000                 0          0                  9   
1        5.920000                 1          0                  3   
2       17.500000                 0          0                  9   
3       19.200000                 1          0                  0   
4        8.930000                 0          1                  1   
5       10.395000                 1          0                  3   
6       13.650000                 1          0                  1   
7       12.857645                 0          0                  5   
8       16.200000                 1          0                  7   
9       19.200000                 1          0                  2   
10      11.800000                 0          0                  9   
11      18.500000                 1          0                  8   
12      15.100000                 1          0                  9   
13      17.600000                 

Here I will identify the feature and target column and split the dataset into training and testing

In [38]:
feature=['Item_Fat_Content','Item_Type','Outlet_Identifier','Outlet_Establishment_Year','Outlet_Size','Outlet_Location_Type','Outlet_Type']
target='sales_bin'

x=dataset[feature]
y=dataset[target]
seed=7
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.4,random_state=4)

In [41]:
def algorithms():
    print('The predicted accuracy scores for the models are as follows:')
    lr=LogisticRegression()
    lr.fit(x_train,y_train)
    lr_prediction=lr.predict(x_test)
    print('LR: {0:4f}'.format(metrics.accuracy_score(y_test,lr_prediction)))

    lda=LinearDiscriminantAnalysis()
    lda.fit(x_train,y_train)
    lda_prediction=lda.predict(x_test)
    print('LDA: {0:4f}'.format(metrics.accuracy_score(y_test,lda_prediction)))

    knn=KNeighborsClassifier()
    knn.fit(x_train,y_train)
    knn_prediction=knn.predict(x_test)
    print('KNN: {0:4f}'.format(metrics.accuracy_score(y_test,knn_prediction)))

    cart=DecisionTreeClassifier()
    cart.fit(x_train,y_train)
    cart_prediction=cart.predict(x_test)
    print('CART: {0:4f}'.format(metrics.accuracy_score(y_test,cart_prediction)))

    nb=GaussianNB()
    nb.fit(x_train,y_train)
    nb_prediction=nb.predict(x_test)
    print('NB: {0:4f}'.format(metrics.accuracy_score(y_test,nb_prediction)))

    svm=SVC()
    svm.fit(x_train,y_train)
    svm_prediction=svm.predict(x_test)
    print('SVM: {0:4f}'.format(metrics.accuracy_score(y_test,svm_prediction)))

In [42]:
algorithms()

The predicted accuracy scores for the models are as follows:
LR: 0.736070
LDA: 0.736070
KNN: 0.703812
CART: 0.737243
NB: 0.320235
SVM: 0.737243


Here I will fit the machine learning model.

In [44]:
svm=SVC()
svm.fit(x_train,y_train)
svm_prediction=svm.predict(x_test)

In [48]:
features=x_test.reset_index()
features=features[['Item_Fat_Content','Item_Type','Outlet_Identifier','Outlet_Establishment_Year','Outlet_Size','Outlet_Location_Type','Outlet_Type']]
actual=(list(y_test))
prediction=(list(svm_prediction))
features['actual']=pd.Series(actual)
features['prediction']=pd.Series(prediction)
final=features

In [55]:
print(list(final['actual']))

[0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 3, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 3, 2, 0, 0, 0, 0, 0, 0, 0, 4, 2, 2, 0, 2, 0, 0, 3, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 0, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 3, 0, 0, 0, 0, 2, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 2, 2, 0, 0, 0, 2, 2, 3, 0, 0, 2, 0, 2, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0, 2, 2, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 3, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 2, 3, 

In [54]:
print(list(final['prediction']))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 

In [57]:
print(final[['actual','prediction']])

      actual  prediction
0          0           0
1          0           0
2          2           0
3          0           0
4          0           0
5          0           0
6          0           0
7          2           0
8          0           0
9          0           0
10         0           0
11         0           0
12         0           0
13         0           0
14         2           0
15         0           0
16         2           0
17         2           0
18         0           0
19         0           0
20         0           0
21         0           0
22         0           0
23         2           0
24         0           0
25         0           0
26         2           0
27         0           0
28         0           0
29         3           0
...      ...         ...
3380       2           0
3381       2           0
3382       0           0
3383       2           0
3384       0           0
3385       0           0
3386       0           0
3387       0           0
